In [ ]:
#  The MIT License (MIT)
#
#  Copyright (c) 2015-2022 Advanced Micro Devices, Inc. All rights reserved.
#
#  Permission is hereby granted, free of charge, to any person obtaining a copy
#  of this software and associated documentation files (the 'Software'), to deal
#  in the Software without restriction, including without limitation the rights
#  to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#  copies of the Software, and to permit persons to whom the Software is
#  furnished to do so, subject to the following conditions:
#
#  The above copyright notice and this permission notice shall be included in
#  all copies or substantial portions of the Software.
#
#  THE SOFTWARE IS PROVIDED 'AS IS', WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#  IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#  FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.  IN NO EVENT SHALL THE
#  AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#  LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#  OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
#  THE SOFTWARE.


# Object Detection with YoloV4
This notebook is intended to be an example of how to use MIGraphX to perform object detection. The model used below is a pre-trained yolov4 from the ONNX model zoo. 

### Download dependencies

In [ ]:
import os.path

if not os.path.exists("./utilities/coco.names"):
    !wget https://github.com/onnx/models/raw/main/validated/vision/object_detection_segmentation/yolov4/dependencies/coco.names -P ./utilities/
if not os.path.exists("./utilities/yolov4_anchors.txt"):
    !wget https://github.com/onnx/models/raw/main/validated/vision/object_detection_segmentation/yolov4/dependencies/yolov4_anchors.txt -P ./utilities/
if not os.path.exists("./utilities/input.jpg"):
    # The image used is from the COCO dataset (https://cocodataset.org/#explore)
    # Other images can be tested by replacing the link below
    image_link = "https://farm3.staticflickr.com/2009/2306189268_88cc86b30f_z.jpg"
    !wget -O ./utilities/input.jpg $image_link
if not os.path.exists("./utilities/yolov4.onnx"):
    !wget https://github.com/onnx/models/raw/main/validated/vision/object_detection_segmentation/yolov4/model/yolov4.onnx -P ./utilities/

### Serialize model using MIGraphX Driver
Please refer to the [MIGraphX Driver example](../../migraphx/migraphx_driver) if you would like more information about this tool.

In [ ]:
if not os.path.exists("yolov4_fp16.mxr"):
    !/opt/rocm/bin/migraphx-driver compile ./utilities/yolov4.onnx --gpu --enable-offload-copy --fp16 --binary -o yolov4_fp16.mxr
if not os.path.exists("yolov4.mxr"):
    !/opt/rocm/bin/migraphx-driver compile ./utilities/yolov4.onnx --gpu --enable-offload-copy --binary -o yolov4.mxr

### Import libraries 
Please refer to [this section](https://github.com/ROCmSoftwarePlatform/AMDMIGraphX#using-migraphx-python-module) of the main README if the migraphx module is not found. 

In [ ]:
import migraphx
import cv2
import time
import numpy as np
import image_processing as ip
from PIL import Image

### Read and pre-process image data

In [ ]:
input_size = 416

original_image = cv2.imread("./utilities/input.jpg")
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
original_image_size = original_image.shape[:2]

image_data = ip.image_preprocess(np.copy(original_image), [input_size, input_size])
image_data = image_data[np.newaxis, ...].astype(np.float32)

### Load and run model

In [ ]:
# Load serialized model (either single- or half-precision)
model = migraphx.load("yolov4.mxr", format="msgpack")
#model = migraphx.load("yolov4_fp16.mxr", format="msgpack")

# Get the name of the input parameter and convert image data to an MIGraphX argument
input_name = next(iter(model.get_parameter_shapes()))
input_argument = migraphx.argument(image_data)

# Evaluate the model and convert the outputs for post-processing
outputs = model.run({input_name: input_argument})
detections = [np.ndarray(shape=out.get_shape().lens(), buffer=np.array(out.tolist()), dtype=float) for out in outputs]

### Post-process the model outputs and display image with detection bounding boxes

In [ ]:
ANCHORS = "./utilities/yolov4_anchors.txt"
STRIDES = [8, 16, 32]
XYSCALE = [1.2, 1.1, 1.05]

ANCHORS = ip.get_anchors(ANCHORS)
STRIDES = np.array(STRIDES)

pred_bbox = ip.postprocess_bbbox(detections, ANCHORS, STRIDES, XYSCALE)
bboxes = ip.postprocess_boxes(pred_bbox, original_image_size, input_size, 0.25)
bboxes = ip.nms(bboxes, 0.213, method='nms')
image = ip.draw_bbox(original_image, bboxes)

image = Image.fromarray(image)
image.show()